In [2]:
import pandas as pd

In [3]:
executions = pd.read_csv("execs_from_fix.csv", dtype={"symbol": "category"})
executions["execution_time"] = pd.to_datetime(executions["execution_time"])
executions["order_time"] = pd.to_datetime(executions["order_time"])
executions.head()

,order_id,order_time,execution_time,symbol,side,order_qty,limit_price,execution_price,exchange
0,ID1542,2025-09-10 08:00:00.377,2025-09-10 08:00:00.509,YANG,1,27,23.70,23.17,ID1516
1,ID1572,2025-09-10 08:00:00.450,2025-09-10 08:00:00.692,IREN,1,30,31.29,31.29,ID1516
2,ID1632,2025-09-10 08:00:00.747,2025-09-10 08:00:00.982,DJT,2,53,16.91,16.91,ID1516
3,ID1711,2025-09-10 08:00:00.866,2025-09-10 08:00:01.006,AMD,2,80,159.56,160.11,ID1516
4,ID1732,2025-09-10 08:00:00.920,2025-09-10 08:00:01.042,SOFI,2,4,25.86,25.89,ID1516


In [4]:
executions.dtypes

order_id                   object
order_time         datetime64[ns]
execution_time     datetime64[ns]
symbol                   category
side                        int64
order_qty                   int64
limit_price               float64
execution_price           float64
exchange                   object
dtype: object

In [5]:
quotes = pd.read_csv("quotes_2025-09-10_small.csv.gz", 
                     dtype = {"ticker": "category"})

quotes["sip_timestamp"] = pd.to_datetime(quotes["sip_timestamp"], unit="ns")

In [6]:
quotes.dtypes

ticker                 category
bid_price               float64
ask_price               float64
bid_size                  int64
ask_size                  int64
bid_exchange              int64
ask_exchange              int64
sip_timestamp    datetime64[ns]
dtype: object

In [7]:
quotes.head()

,ticker,bid_price,ask_price,bid_size,ask_size,bid_exchange,ask_exchange,sip_timestamp
0,A,0.00,0.0,0,0,19,19,2025-09-10 07:59:00.001962082
1,A,0.00,0.0,0,0,8,8,2025-09-10 07:59:00.017042971
2,A,0.00,0.0,0,0,20,20,2025-09-10 08:00:00.000466709
3,A,95.18,0.0,1,0,8,8,2025-09-10 08:00:00.010976770
4,A,98.25,0.0,1,0,8,8,2025-09-10 08:00:00.011016840


In [8]:
executions.shape

(493235, 9)

In [8]:
all_tickers = set(executions["symbol"].cat.categories).union(set(quotes["ticker"].cat.categories))

executions.symbol = executions.symbol.cat.set_categories(all_tickers)
quotes.ticker = quotes.ticker.cat.set_categories(all_tickers)

In [57]:
TICKERS = ["AAPL", "WMT"]
ex_sample = executions.loc[executions.symbol.isin(TICKERS)]
q_sample = quotes.loc[quotes.ticker.isin(TICKERS)]
ex_sample.shape, q_sample.shape

((513, 9), (1163558, 8))

In [58]:
ex_sample = ex_sample.sort_values(by=["order_time", "symbol"])
q_sample = q_sample.sort_values(by=["sip_timestamp", "ticker"])

In [59]:
ex_sample = pd.merge_asof(
    ex_sample,
    q_sample,
    left_on='order_time',
    right_on='sip_timestamp',
    left_by='symbol',
    right_by='ticker',
    direction='backward'
)

ex_sample.tail()

,order_id,order_time,execution_time,symbol,side,order_qty,limit_price,execution_price,exchange,ticker,bid_price,ask_price,bid_size,ask_size,bid_exchange,ask_exchange,sip_timestamp
508,ID653629,2025-09-10 22:54:09.245,2025-09-10 22:56:07.496,AAPL,1,11,227.36,227.36,ID1516,AAPL,227.34,227.38,1,6,11,11,2025-09-10 22:53:55.375620889
509,ID658151,2025-09-10 23:22:13.801,2025-09-10 23:26:21.300,AAPL,1,10,227.14,227.14,ID1516,AAPL,227.12,227.21,1,1,12,12,2025-09-10 23:22:09.431537988
510,ID659195,2025-09-10 23:28:52.600,2025-09-10 23:32:24.020,AAPL,1,10,227.13,227.13,ID1516,AAPL,227.13,227.15,1,3,11,8,2025-09-10 23:28:42.851894426
511,ID659971,2025-09-10 23:33:53.907,2025-09-10 23:42:47.670,AAPL,1,100,227.12,227.12,ID1516,AAPL,227.11,227.15,1,2,12,8,2025-09-10 23:33:32.126953834
512,ID663123,2025-09-10 23:52:42.932,2025-09-10 23:52:43.036,AAPL,2,3,227.00,227.00,ID1516,AAPL,227.00,227.01,20,1,12,12,2025-09-10 23:52:26.759078036


In [60]:
ex_sample = ex_sample.dropna()
ex_sample.head()



,order_id,order_time,execution_time,symbol,side,order_qty,limit_price,execution_price,exchange,ticker,bid_price,ask_price,bid_size,ask_size,bid_exchange,ask_exchange,sip_timestamp
0,ID3988,2025-09-10 08:00:07.549,2025-09-10 08:28:40.937,AAPL,1,3,233.54,233.54,ID1516,AAPL,234.00,234.30,2,2,8,11,2025-09-10 08:00:07.333331695
1,ID4950,2025-09-10 08:00:11.654,2025-09-10 08:28:40.949,AAPL,1,3,233.54,233.54,ID1516,AAPL,234.00,234.30,2,7,8,11,2025-09-10 08:00:10.790205093
2,ID5529,2025-09-10 08:00:14.171,2025-09-10 08:28:40.949,AAPL,1,1,233.54,233.54,ID1516,AAPL,234.00,234.29,2,1,8,11,2025-09-10 08:00:14.033499328
3,ID8195,2025-09-10 08:01:56.292,2025-09-10 08:02:05.550,AAPL,1,10,234.17,234.17,ID1516,AAPL,234.22,234.25,14,1,11,11,2025-09-10 08:01:56.101296762
4,ID14463,2025-09-10 08:06:48.677,2025-09-10 08:10:19.403,AAPL,1,41,233.86,233.86,ID1516,AAPL,233.86,234.00,5,1,12,12,2025-09-10 08:06:23.104518591


In [ ]:
ex_sample['price_improvement'] = ((ex_sample['limit_price'] - ex_sample['execution_price'])*(ex_sample['side'] == 1) + 
                                (ex_sample['execution_price'] - ex_sample['limit_price'])*(ex_sample['side'] != 1))

,order_id,order_time,execution_time,symbol,side,order_qty,limit_price,execution_price,exchange,ticker,bid_price,ask_price,bid_size,ask_size,bid_exchange,ask_exchange,sip_timestamp,price_improvement


In [ ]:
(ex_sample["execution_price"] == 0).sum()

np.int64(0)

In [ ]:

ex_sample.loc[ex_sample.sip_timestamp >= "2025-09-10 08:00"]

,order_id,order_time,execution_time,symbol,side,order_qty,limit_price,execution_price,exchange,ticker,bid_price,ask_price,bid_size,ask_size,bid_exchange,ask_exchange,sip_timestamp,price_improvement
0,ID3988,2025-09-10 08:00:07.549,2025-09-10 08:28:40.937,AAPL,1,3,233.54,233.54,ID1516,AAPL,234.00,234.30,2,2,8,11,2025-09-10 08:00:07.333331695,0.0
1,ID4950,2025-09-10 08:00:11.654,2025-09-10 08:28:40.949,AAPL,1,3,233.54,233.54,ID1516,AAPL,234.00,234.30,2,7,8,11,2025-09-10 08:00:10.790205093,0.0
2,ID5529,2025-09-10 08:00:14.171,2025-09-10 08:28:40.949,AAPL,1,1,233.54,233.54,ID1516,AAPL,234.00,234.29,2,1,8,11,2025-09-10 08:00:14.033499328,0.0
3,ID8195,2025-09-10 08:01:56.292,2025-09-10 08:02:05.550,AAPL,1,10,234.17,234.17,ID1516,AAPL,234.22,234.25,14,1,11,11,2025-09-10 08:01:56.101296762,0.0
4,ID14463,2025-09-10 08:06:48.677,2025-09-10 08:10:19.403,AAPL,1,41,233.86,233.86,ID1516,AAPL,233.86,234.00,5,1,12,12,2025-09-10 08:06:23.104518591,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
508,ID653629,2025-09-10 22:54:09.245,2025-09-10 22:56:07.496,AAPL,1,11,227.36,227.36,ID1516,AAPL,227.34,227.38,1,6,11,11,2025-09-10 22:53:55.375620889,0.0
509,ID658151,2025-09-10 23:22:13.801,2025-09-10 23:26:21.300,AAPL,1,10,227.14,227.14,ID1516,AAPL,227.12,227.21,1,1,12,12,2025-09-10 23:22:09.431537988,0.0
510,ID659195,2025-09-10 23:28:52.600,2025-09-10 23:32:24.020,AAPL,1,10,227.13,227.13,ID1516,AAPL,227.13,227.15,1,3,11,8,2025-09-10 23:28:42.851894426,0.0
511,ID659971,2025-09-10 23:33:53.907,2025-09-10 23:42:47.670,AAPL,1,100,227.12,227.12,ID1516,AAPL,227.11,227.15,1,2,12,8,2025-09-10 23:33:32.126953834,0.0


In [9]:
quotes = quotes.loc[(quotes.sip_timestamp >= "2025-09-10 09:30") & (quotes.sip_timestamp <= "2025-09-10 16:00")]

In [10]:
quotes.shape

(173187382, 8)

In [ ]:
executions = executions.sort_values(by=["order_time", "symbol"])
quotes = quotes.sort_values(by=["sip_timestamp", "ticker"])

merged = pd.merge_asof(
    executions,
    quotes,
    left_on='order_time',
    right_on='sip_timestamp',
    left_by='symbol',
    right_by='ticker',
    direction='backward'
)

ValueError: left keys must be sorted

In [ ]:
import time
